# IEEE LV Powerflow Solver Test

## Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/IEEE_EU_LV/IEEE_EU_LV_reduced/'
filename = 'IEEE_EU_LV'
download_grid_data(filename+'_EQ.xml', url+'Rootnet_FULL_NE_13J16h_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'Rootnet_FULL_NE_13J16h_TP.xml')
download_grid_data(filename+'_SV.xml', url+'Rootnet_FULL_NE_13J16h_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import villas.dataprocessing.validationtools as validationtools
import dpsimpy

### Run simulation

In [ ]:
sim_name = 'IEEE_EU_LV_reduced'

In [ ]:
reader = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.info, dpsimpy.LogLevel.off)
system = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.NONE)

logger = dpsimpy.Logger(sim_name)
for node in system.nodes:
    logger.log_attribute(node.name() + '.V', 'v', node)

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(1.0)
sim.set_final_time(1.0)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(True)

sim.add_logger(logger)
sim.run()


## Read DPsim results

In [ ]:
path = 'logs/'
logName = 'IEEE_EU_LV_reduced'
dpsim_result_file = path + logName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

# Fix for dpsim naming - TODO: unify dpsim notation in log file and update villas-dataprocessing accordingly
for ts,values in ts_dpsim.items():
    values.name = values.name[:-2]

## Read NEPLAN results as reference

In [ ]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/Neplan/ReferenceGrids/IEEE_EU_LV.rlf'
neplan_result_file = 'reference-results/IEEE_EU_LV.rlf'
urllib.request.urlretrieve(url, neplan_result_file) 
print(neplan_result_file)

ts_NEPLAN = read_timeseries_NEPLAN_loadflow(neplan_result_file)
# print([ts_NEPLAN[i].name for i in range(len(ts_NEPLAN))])

## Comparison and assertion of results

In [ ]:
net_name='IEEE_EU_LV_reduced'
threshold=0.5

ts_NEPLAN_standardized=validationtools.convert_neplan_to_standard_timeseries(ts_NEPLAN)
#print([[ts_NEPLAN_standardized[i].name, ts_NEPLAN_standardized[i].values] for i in range(len(ts_NEPLAN_standardized))])
ts_dpsim_standardized=validationtools.convert_dpsim_to_standard_timeseries(ts_dpsim)
#print([[ts_dpsim_standardized[i].name, ts_dpsim_standardized[i].values] for i in range(len(ts_dpsim_standardized))])
res_err=validationtools.compare_timeseries(ts_NEPLAN_standardized,ts_dpsim_standardized)
validationtools.assert_modelica_results(net_name,res_err,threshold)